In [41]:
import zarr
import xarray
import s3fs
import os
from datetime import datetime

In [42]:
ds = xarray.open_dataset(
    "/home/havardf/Downloads/arome_arctic_pp_2_5km_latest.nc",    
    decode_times=False,
)

In [43]:
#ds
ref_time_epoch = ds["forecast_reference_time"].values.cumsum()[0]
ref_time = datetime.fromtimestamp(ref_time_epoch)

In [44]:
# Drop variables that var 0 dimensions as upload fails when trying to upload them.
ds = ds.drop_vars("forecast_reference_time")
ds = ds.drop_vars("projection_lambert")

# Compare the data if needed
#compressor = zarr.Blosc(cname='zstd', clevel=3)
#encoding = {vname: {'compressor': compressor} for vname in ds.data_vars}
# Save to zarr
# ds.to_zarr(store='zarr_example', group="/root", encoding=encoding, consolidated=True)

In [45]:
ds

<xarray.Dataset>
Dimensions:                               (height0: 1, height1: 1, height3: 1, height_above_msl: 1, p0: 1, time: 67, x: 719, y: 929)
Coordinates:
  * time                                  (time) float64 1.605e+09 ... 1.605e+09
  * height_above_msl                      (height_above_msl) float32 0.0
  * height0                               (height0) float32 0.0
  * height1                               (height1) float32 2.0
  * height3                               (height3) float32 10.0
  * x                                     (x) float32 303603.16 ... 2098603.2
  * y                                     (y) float32 -872931.56 ... 1447068.4
    longitude                             (y, x) float64 ...
    latitude                              (y, x) float64 ...
  * p0                                    (p0) float32 1013.25
Data variables:
    surface_air_pressure                  (time, height0, y, x) float32 ...
    relative_humidity_2m                  (time, height1, y, x) float32 ...
    x_wind_10m                            (time, height3, y, x) float32 ...
    y_wind_10m                            (time, height3, y, x) float32 ...
    air_pressure_at_sea_level             (time, height_above_msl, y, x) float32 ...
    precipitation_amount_acc              (time, height0, y, x) float32 ...
    wind_speed_of_gust                    (time, height3, y, x) float32 ...
    fog_area_fraction                     (time, height0, y, x) float32 ...
    land_area_fraction                    (y, x) float32 ...
    precipitation_amount                  (time, y, x) float32 ...
    precipitation_amount_high_estimate    (time, y, x) float32 ...
    precipitation_amount_low_estimate     (time, y, x) float32 ...
    precipitation_amount_middle_estimate  (time, y, x) float32 ...
    precipitation_amount_prob_low         (time, y, x) float32 ...
    cloud_area_fraction                   (time, y, x) float32 ...
    high_type_cloud_area_fraction         (time, y, x) float32 ...
    low_type_cloud_area_fraction          (time, y, x) float32 ...
    medium_type_cloud_area_fraction       (time, y, x) float32 ...
    helicopter_triggered_index            (time, y, x) float32 ...
    number_of_tracks                      (time, y, x) float32 ...
    strike_probability                    (time, y, x) float32 ...
    air_temperature_2m                    (time, height1, y, x) float32 ...
    altitude                              (y, x) float32 ...
Attributes:
    Conventions:         CF-1.6
    institution:         Norwegian Meteorological Institute, MET Norway
    creator_url:         met.no
    summary:             AROME_Arctic is a convection-permitting atmosphere m...
    title:               AROME_Arctic 2.5km
    min_time:            2020-11-10 09:00:00Z
    max_time:            2020-11-13
    geospatial_lat_min:  62.0
    geospatial_lat_max:  88.0
    geospatial_lon_min:  -18.0
    geospatial_lon_max:  80.0
    references:          unknown
    license:             https://www.met.no/frie-meteorologiske-data/lisensie...
    comment:             none
    history:             2020-11-10 creation by fimex
    source:              arome_arctic

In [51]:
%%time
# AWS S3 path
#os.environ['AWS_CONFIG_FILE'] = '/home/havardf/git/s-enda/dataaccess/client/config/aws_config.ini'
s3_path = 's3://aromearctic/' + dt.strftime("%Y%m%dT%H00Z") 
# Initilize the S3 file system
s3 = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "http://localhost:9000"},key="test", secret="testtesttest1")
store = s3fs.S3Map(root=s3_path, s3=s3, check=False)
# Compression
#compressor = zarr.Blosc(cname='zstd', clevel=3)
#encoding = {vname: {'compressor': compressor} for vname in ds.data_vars}
# Save to zarr
ds.to_zarr(store=store, consolidated=True)

CPU times: user 1min, sys: 3.8 s, total: 1min 3s
Wall time: 1min 44s
